# What has been done so far?

 - A more documented READme file. However, it is not complete and STILL NEEDS EDITING.
 - A supposed list of Ensembl IDs has been obtained but it only includes animalia, no plants.
 - The naming of 3 species on Knetminer is identified, simply lower case of scientific name with _ for space. However, they don't seem to be from actual EBI database but elsewhere that EBI links. This is based on the fact that the names below works for URL requests
     - rice is oryza_sativa
     - wheat is triticum_aestivum
     - arabidopsis is arabidopsis_thaliana
 - A text file is now the input needed to generate keywords for knetminer, no harding coding required.
 - The script directly reads from the csv instead of a formated_results.txt which has been removed.
 - Genome.json output is removed.
 - Distance is now a parameter instead of 1000bp hard coded.

# practical actions to take next

 - Report findings on Ensembl species to Keywan. Inform him that the rice, wheat and arabidopsis database might be from NCBI.
 
 
 - Read python documentation for argparse so you can make arguments such as -p, -d etc optional with a default value. https://towardsdatascience.com/learn-enough-python-to-be-useful-argparse-e482e1764e05 (concise version)
 
 
 - Find the names of arabidopsisknet. If too difficult, ask Ajit for the urls
     - http://www.gregreda.com/2015/02/15/web-scraping-finding-the-api/
     - the url for wheatknet is http://knetminer.rothamsted.ac.uk/wheatknet/genome?
     - url for arabidopsis is unknown.
 
 
 - Use a Jupyter smalltest note to check if you are getting the functional annotations for each gene correctly from ensembl instead of just "NONE"
 
 
 - Think logically and brainstorm on a way to merge information in summary_gene_discovered and kent_summary (url and score).
 
 
 - If Keywan agrees, state the 3 species names and use argparse to make them a flexible parameter.

# Experimentation with concatenating 2 summary files

In [16]:
filtered_snps="T:\colin\BRice\gwas-gene-discoveryV2\MLM.blupWidth.GWAS.Results\filtered_snps.txt"
disc_genes="T:\colin\BRice\gwas-gene-discoveryV2\MLM.blupWidth.GWAS.Results\summary_genes_discovered.txt"
args="T:\colin\BRice\gwas-gene-discoveryV2\MLM.blupWidth.GWAS.Results\mock_keyword_list.txt"
genetab="T:\colin\BRice\gwas-gene-discoveryV2\MLM.blupWidth.GWAS.Results\genetable.txt"

##### The solution might be write a super function that has all the aspects of 3 functions: summary, knetapi and  knetsummary

In [18]:
def knetapi(disc_genes, genetab):
    '''Use the genes from the summary file to search for Knetminer genome api.'''
    with open(disc_genes, "r") as fs:
        next(fs)
        with open(genetab, "w") as fj:
            genes = []
            with open(args.l, "r") as fl:
                pheno = []
                for line in fs:
                    col = line.split("\t")
                    genes.append(col[0])
                genelist = (",").join(genes) #join all iterative elements by ,
                #print(genelist)
                for line in fl:
                    pheno.append(line.rstrip())
                    print(pheno)
                #use str.join() to convert multiple elments in a list into one string.
                keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
                link = "http://knetminer.rothamsted.ac.uk/riceknet/genome?"
                parameters = {"keyword":keyw1, "list":genelist}
                r = requests.get(link, params=parameters)
                
                #check if request is succesfsul.
                if not r.ok:
                    r.raise_for_status()
                    sys.exit()
                
                #printing out genetable to a file.
                decoded = str(r.json()[u'geneTable'])
                print(decoded)